[View in Colaboratory](https://colab.research.google.com/github/teddylee777/machine-learning/blob/master/Tensorflow/05_CNN_with_mnist_dataset.ipynb)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [0]:
# parameter
learning_rate = 0.01
batch_size = 1000
drop_out = 0.7
training_epoch = 20

In [0]:
X_train = mnist.train.images
y_train = mnist.train.labels

In [0]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

In [0]:
X_input = tf.reshape(X, shape=[-1, 28, 28, 1])

In [0]:
W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]), tf.float32)
L1 = tf.nn.conv2d(X_input, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

W2 = tf.Variable(tf.random_normal([5, 5, 32, 64]), tf.float32)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

L2 = tf.layers.flatten(L2)

W3 = tf.Variable(tf.random_normal([7*7*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))
L3 = tf.matmul(L2, W3) + b3
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([1024, 10]))
b4 = tf.Variable(tf.random_normal([10]))
L4 = tf.matmul(L3, W4) + b4

hypothesis = tf.nn.softmax(L4)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=L4, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(hypothesis, axis=1)
actual = tf.argmax(Y, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, actual), tf.float32))

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)
summaries = tf.summary.merge_all()

In [9]:
!cat /proc/meminfo

MemTotal:       13335188 kB
MemFree:         1265416 kB
MemAvailable:   12052480 kB
Buffers:          153976 kB
Cached:         10339976 kB
SwapCached:            0 kB
Active:          1046548 kB
Inactive:       10268096 kB
Active(anon):     741148 kB
Inactive(anon):    82584 kB
Active(file):     305400 kB
Inactive(file): 10185512 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1632 kB
Writeback:             0 kB
AnonPages:        820756 kB
Mapped:           233464 kB
Shmem:            254968 kB
Slab:             660376 kB
SReclaimable:     622372 kB
SUnreclaim:        38004 kB
KernelStack:        3648 kB
PageTables:         6120 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667592 kB
Committed_AS:    2486404 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('logs/05_Convolution_Neural_Network_With_Mnist_Dataset', sess.graph)
    num_batch = int(mnist.train.num_examples / batch_size)
    for epoch in range(training_epoch):
        avg_cost = 0
        for b in range(num_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _, accuracy_val, summary_ = sess.run([cost, optimizer, accuracy, summaries], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: drop_out})
            avg_cost += cost_val / num_batch
        print("epoch {0}, cost = {1:.5f}, accuracy = {2:.5f}".format(epoch, cost_val, accuracy_val))
        summary_writer.add_summary(summary_, global_step=epoch)
    accuracy_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}) 
    print("result, accuracy = {:.5f}".format(accuracy_val))